In [1]:
import os
from time import time
from feature_extraction import __get

path = r'/Users/loewi/Documents/Pre_Learn/classification/20news-bydate/'
os.chdir(path)
#print(os.getcwd())

t0 = time() 

newsgroups_train = __get('20news-bydate-train')
newsgroups_test = __get('20news-bydate-test')

duration = time() - t0
print('%0.2fs get data package ：）'%duration)

X_train, X_test = newsgroups_train['data'], newsgroups_test['data']
y_train, y_test = newsgroups_train['docs'], newsgroups_test['docs']
print('X & y prepared')

11.09s get data package ：）
X & y prepared


In [2]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,HashingVectorizer,TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score,StratifiedKFold,GridSearchCV,RandomizedSearchCV
from pprint import pprint
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics

In [12]:
vectorizer = HashingVectorizer(stop_words='english')
X = vectorizer.fit_transform( X_train )
Xt= vectorizer.transform( X_test )
clf = RandomForestClassifier()
clf.fit(X, y_train)
#RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=2, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)
#print(clf.feature_importances_)

pred = clf.predict(Xt)
score = metrics.accuracy_score(y_test, pred)
print("accuracy:   %0.3f" % score)

accuracy:   0.505


In [20]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b'
                        )),
                
                ('clf', RandomForestClassifier()),
                ])
parameters = {
          #  'vect__ngram_range':((1, 1), (1, 2)), 
            'clf__n_estimators': range(10,201,10), #弱学习器迭代次数
}    
#n_iter_search = 20
#cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = RandomizedSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = 5)
print("Performing randomized search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))    

Performing randomized search...
pipeline: ['vect', 'clf']
parameters:
{'clf__n_estimators': range(10, 201, 10)}
Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 84.3min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 113.1min finished


done in 7912.154s

Best score: 0.782
Best parameters set:
	clf__n_estimators: 180


In [34]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                
                ('clf', RandomForestClassifier(
                        
                )),
                ])
parameters = {
            'clf__n_estimators': range(150,201,10),
            'clf__max_depth':range(80,101,10),
            'clf__min_samples_split':range(110,151,10), 

}    
#n_iter_search = 20
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = cv)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))    

Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__max_depth': range(80, 101, 10),
 'clf__min_samples_split': range(110, 151, 10),
 'clf__n_estimators': range(190, 201, 10)}
Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.4min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed: 13.7min finished


done in 852.781s

Best score: 0.743
Best parameters set:
	clf__max_depth: 100
	clf__min_samples_split: 110
	clf__n_estimators: 190


In [39]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                
                ('clf', RandomForestClassifier(
                        n_estimators = 190,
                        max_depth = 100,
                    
                )),
                ])
#内部节点再划分
parameters = {
            'clf__min_samples_split':range(10,201,10),
            'clf__min_samples_leaf':range(10,101,10)

}    
#n_iter_search = 100
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = cv)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))    

Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__min_samples_leaf': range(10, 101, 10),
 'clf__min_samples_split': range(10, 201, 10)}
Fitting 5 folds for each of 200 candidates, totalling 1000 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  6.9min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 13.4min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done 1000 out of 1000 | elapsed: 26.7min finished


done in 1612.833s

Best score: 0.698
Best parameters set:
	clf__min_samples_leaf: 10
	clf__min_samples_split: 160


In [40]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                
                ('clf', RandomForestClassifier(
                        n_estimators = 180,                        
                        min_samples_split = 160,
                        min_samples_leaf = 10
                )),
                ])
#内部节点再划分
parameters = {
            'clf__max_depth':range(2,202,10),
            

}    
#n_iter_search = 100
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = GridSearchCV(pipeline, parameters,n_jobs=-1, verbose=1, cv = cv)
print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))    


Performing grid search...
pipeline: ['vect', 'clf']
parameters:
{'clf__max_depth': range(2, 202, 10)}
Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  3.9min finished


done in 246.508s

Best score: 0.697
Best parameters set:
	clf__max_depth: 152


In [53]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                
                ('clf', RandomForestClassifier(
                        n_estimators = 180,
                )),
                ])
parameters = {
            'clf__max_depth':range(2,502,20),#决策树最大深度
            'clf__min_samples_split':range(2,501,20),#内部节点再划分所需最小样本数
            'clf__min_samples_leaf': range(1, 101, 10),
}    
# bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=2, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=0, verbose=0, warm_start=False

n_iter_search = 500
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = RandomizedSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv = cv, n_iter = n_iter_search)
print("Performing randomized search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))    

Performing randomized search...
pipeline: ['vect', 'clf']
parameters:
{'clf__max_depth': range(2, 502, 20),
 'clf__min_samples_leaf': range(1, 101, 10),
 'clf__min_samples_split': range(2, 501, 20)}
Fitting 5 folds for each of 500 candidates, totalling 2500 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:  3.6min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 11.7min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed: 23.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed: 60.5min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed: 79.2min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 102.0min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 134.4min
[Parallel(n_jobs=-1)]: Done 2500 out of 2500 | elapsed: 136.6min finished


done in 9124.378s

Best score: 0.788
Best parameters set:
	clf__max_depth: 422
	clf__min_samples_leaf: 1
	clf__min_samples_split: 2


In [62]:
import numpy as np
def report(results, n_top=10):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
            
report(clf.cv_results_)           

Model with rank: 1
Mean validation score: 0.788 (std: 0.008)
Parameters: {'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_depth': 422}

Model with rank: 2
Mean validation score: 0.783 (std: 0.008)
Parameters: {'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_depth': 162}

Model with rank: 3
Mean validation score: 0.781 (std: 0.007)
Parameters: {'clf__min_samples_split': 2, 'clf__min_samples_leaf': 1, 'clf__max_depth': 142}

Model with rank: 4
Mean validation score: 0.778 (std: 0.005)
Parameters: {'clf__min_samples_split': 22, 'clf__min_samples_leaf': 1, 'clf__max_depth': 202}

Model with rank: 5
Mean validation score: 0.772 (std: 0.006)
Parameters: {'clf__min_samples_split': 22, 'clf__min_samples_leaf': 1, 'clf__max_depth': 142}

Model with rank: 6
Mean validation score: 0.772 (std: 0.006)
Parameters: {'clf__min_samples_split': 42, 'clf__min_samples_leaf': 1, 'clf__max_depth': 202}

Model with rank: 7
Mean validation score: 0.766 (std: 0.010)
Parame

In [ ]:
pipeline = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                
                ('clf', RandomForestClassifier(
                        n_estimators = 180,
                    
                )),
                ])
parameters = {
            'clf__max_depth':range(150,502,10),#决策树最大深度
            'clf__min_samples_split':range(2,101,10),#内部节点再划分所需最小样本数
            'clf__max_features':('sqrt', 0.8),#划分时考虑的最大特征数
            'clf__min_samples_leaf': range(10,101,10),#限制叶子节点最少的样本数;叶子节点数目小于样本数，会和兄弟节点一起被剪枝
            
}    
# bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=2, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=0, verbose=0, warm_start=False

n_iter_search = 100
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
clf = RandomizedSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv = cv, n_iter = n_iter_search)
print("Performing randomized search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
clf.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % clf.best_score_)
print("Best parameters set:")
best_parameters = clf.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))    

Performing randomized search...
pipeline: ['vect', 'clf']
parameters:
{'clf__max_depth': range(150, 502, 10),
 'clf__max_features': ('sqrt', 0.8),
 'clf__min_samples_leaf': range(10, 101, 10),
 'clf__min_samples_split': range(2, 101, 10)}
Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed: 124.4min
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed: 562.2min


In [3]:
#%% predict
    
from sklearn import metrics  
clf_validated = Pipeline([
                ('vect', HashingVectorizer(
                        lowercase = True,
                        stop_words= 'english',
                        token_pattern= r'(?u)\b[a-zA-Z_][a-zA-Z_]+\b',
                        )),
                
                ('clf', RandomForestClassifier(
                        n_estimators = 180,
                        min_samples_split = 22,
                        max_depth = 202,
                ))
])
                
clf_validated.fit(X_train, y_train)  
cv = StratifiedKFold(n_splits= 5 ,random_state=0,shuffle=True)
scores = cross_val_score(clf_validated, X_train, y_train,  cv = cv, scoring='accuracy')
print (scores)
score = scores.mean()
print("training accuracy: %0.3f (+/- %0.3f)" % (score, scores.std() * 2))
              
pred = clf_validated.predict(X_test)
score = metrics.accuracy_score(y_test, pred)
print("testing accuracy:   %0.3f" % score)


[0.64477743 0.64872021 0.66961131 0.6659292  0.65631929]
training accuracy: 0.657 (+/- 0.019)
testing accuracy:   0.616


In [ ]:

#RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
#            max_depth=2, max_features='auto', max_leaf_nodes=None,
#            min_impurity_decrease=0.0, min_impurity_split=None,
#            min_samples_leaf=1, min_samples_split=2,
#            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
#            oob_score=False, random_state=0, verbose=0, warm_start=False)